In [2]:
import pandas as pd
df=pd.read_csv('annotated_teluk asmara.csv')

In [3]:
df=df.dropna()
df

,user,waktu,ulasan,sentiment,aspect
0,SAM RIJAL,1 hari lalu,"Luar biasa, pantainya sangat indah",Positive,Pemandangan
1,Yuda Sukur Pangestu,seminggu lalu,Ada penyewaan karpet atau perlengkapan lainnya...,Positive,Fasilitas
2,Wachid Aziz,seminggu lalu,bagus viewnya indah,Positive,Pemandangan
4,Imroatul Azizah,seminggu lalu,Seru banget....pantainya bersahabat pemandanga...,Positive,Pemandangan
5,sumarlik mala,seminggu lalu,"Pantai yang sangat bersih menurut saya,pasir p...",Positive,Akses
...,...,...,...,...,...
593,juan joe,2 tahun lalu,dari pos loket ke parkiran..jalan rusak parah....,Negative,Akses
594,Amri 04,2 tahun lalu,"raja ampatnya jawa nih boss, asyik aja camp di...",Positive,Pemandangan
596,CINTA JAYA MAKMUR CJM,2 tahun lalu,Bagus asri dan sangat bersih,Positive,Pemandangan
597,Suelmi Widjilingtyas,2 tahun lalu,"Pantai cantik, ada villa dengan berbagai ukura...",Negative,Akses


In [4]:

df = df.rename(columns={'sentiment': 'label','ulasan':'text'})

In [5]:

df = df.reindex(columns=['text','label'])

In [6]:
df

,text,label
0,"Luar biasa, pantainya sangat indah",Positive
1,Ada penyewaan karpet atau perlengkapan lainnya...,Positive
2,bagus viewnya indah,Positive
4,Seru banget....pantainya bersahabat pemandanga...,Positive
5,"Pantai yang sangat bersih menurut saya,pasir p...",Positive
...,...,...
593,dari pos loket ke parkiran..jalan rusak parah....,Negative
594,"raja ampatnya jawa nih boss, asyik aja camp di...",Positive
596,Bagus asri dan sangat bersih,Positive
597,"Pantai cantik, ada villa dengan berbagai ukura...",Negative


In [7]:

from sklearn.model_selection import train_test_split

df_trainVal, df_test = train_test_split(df, shuffle=True, test_size=0.2)


In [8]:
df_trainVal.shape

(376, 2)

In [9]:
df_train, df_val = train_test_split(df_trainVal, shuffle=True, test_size=0.3)

In [10]:

df_train.shape

(263, 2)

In [11]:

df_train.to_csv("train.csv", index=False)
df_val.to_csv("eval.csv", index=False)
df_test.to_csv("test.csv", index=False)

In [12]:
!pip install transformers

In [13]:

#Clone repositori berisi dataset dan model indobenchmark untuk proses NLP
!git clone https://github.com/indobenchmark/indonlu.git


Cloning into 'indonlu'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 500 (delta 115), reused 139 (delta 110), pack-reused 316
Receiving objects: 100% (500/500), 9.45 MiB | 11.89 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [14]:
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
import torch
import random
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
from indonlu.utils.forward_fn import forward_sequence_classification
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [15]:
def document_sentiment_metrics_fn(list_hyp, list_label):
    metrics = {}
    metrics["ACC"] = accuracy_score(list_label, list_hyp)
    metrics["F1"] = f1_score(list_label, list_hyp, average='macro', zero_division = 1)
    metrics["REC"] = recall_score(list_label, list_hyp, average='macro', zero_division = 1)
    metrics["PRE"] = precision_score(list_label, list_hyp, average='macro', zero_division = 1)
    return metrics

In [16]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)


In [19]:
import numpy as np

In [20]:

# Set random seed
set_seed(123)


In [21]:

# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p2')
config.num_labels = 2

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p2', config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import numpy as np
from torch.utils.data import Dataset, DataLoader

class SejutaDataset(Dataset):
    # Static constant variable
    LABEL2INDEX = {'Negative': 0, 'Positive': 1}
    INDEX2LABEL = {0: 'Negative', 1: 'Positive'}
    NUM_LABELS = 2

    def load_dataset(self, path):
        # Load dataset
        dataset = pd.read_csv(path)
        dataset['label'] = dataset['label'].apply(lambda sen: self.LABEL2INDEX[sen])
        return dataset

    def __init__(self, dataset_path, tokenizer, no_special_token=False, *args, **kwargs):
        self.data = self.load_dataset(dataset_path)
        self.tokenizer = tokenizer
        self.no_special_token = no_special_token

    def __getitem__(self, index):
        text, label = self.data.loc[index,'text'], self.data.loc[index,'label']
        subwords = self.tokenizer.encode(text, add_special_tokens=not self.no_special_token)
        return np.array(subwords), np.array(label), text

    def __len__(self):
        return len(self.data)

class SejutaDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(SejutaDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = self._collate_fn
        self.max_seq_len = max_seq_len

    def _collate_fn(self, batch):
        batch_size = len(batch)
        max_seq_len = max(map(lambda x: len(x[0]), batch))
        max_seq_len = min(self.max_seq_len, max_seq_len)

        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        label_batch = np.full((batch_size, 1), -100, dtype=np.int64)

        seq_list = []
        for i, (subwords, label, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            label_batch[i] = label

            seq_list.append(raw_seq)

        return subword_batch, mask_batch, label_batch, seq_list

In [23]:
train_dataset_path = '/content/train.csv'
valid_dataset_path = '/content/eval.csv'
test_dataset_path = '/content/test.csv'

In [24]:
train_dataset = SejutaDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = SejutaDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = SejutaDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = SejutaDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=2, shuffle=True)
valid_loader = SejutaDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=2, shuffle=False)
test_loader = SejutaDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=2, shuffle=False)


In [27]:

w2i, i2w = SejutaDataset.LABEL2INDEX, SejutaDataset.INDEX2LABEL
print(w2i)
print(i2w)


{'Negative': 0, 'Positive': 1}
{0: 'Negative', 1: 'Positive'}


In [25]:
optimizer = optim.Adam(model.parameters(), lr=5e-6)
model = model.cuda()

In [28]:
# Train
n_epochs = 7
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

(Epoch 1) TRAIN LOSS:0.6327 LR:0.00000500: 100%|██████████| 9/9 [00:06<00:00,  1.36it/s]


(Epoch 1) TRAIN LOSS:0.6327 ACC:0.58 F1:0.48 REC:0.52 PRE:0.54 LR:0.00000500


VALID LOSS:0.5733 ACC:0.69 F1:0.60 REC:0.62 PRE:0.76: 100%|██████████| 4/4 [00:00<00:00,  4.70it/s]


(Epoch 1) VALID LOSS:0.5733 ACC:0.69 F1:0.60 REC:0.62 PRE:0.76


(Epoch 2) TRAIN LOSS:0.5176 LR:0.00000500: 100%|██████████| 9/9 [00:04<00:00,  1.89it/s]


...

(Epoch 6) VALID LOSS:0.2422 ACC:0.90 F1:0.90 REC:0.90 PRE:0.90


(Epoch 7) TRAIN LOSS:0.1011 LR:0.00000500: 100%|██████████| 9/9 [00:03<00:00,  2.28it/s]


(Epoch 7) TRAIN LOSS:0.1011 ACC:0.99 F1:0.99 REC:0.99 PRE:0.99 LR:0.00000500


VALID LOSS:0.2334 ACC:0.89 F1:0.89 REC:0.89 PRE:0.89: 100%|██████████| 4/4 [00:00<00:00,  5.38it/s]

(Epoch 7) VALID LOSS:0.2334 ACC:0.89 F1:0.89 REC:0.89 PRE:0.89


In [29]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df_pred = pd.DataFrame({'label':list_hyp}).reset_index()
df_pred.to_csv('pred.txt', index=False)


100%|██████████| 3/3 [00:01<00:00,  2.85it/s]


In [30]:
df_pred

,index,label
0,0,Negative
1,1,Negative
2,2,Positive
3,3,Negative
4,4,Negative
...,...,...
90,90,Positive
91,91,Positive
92,92,Negative
93,93,Positive


In [31]:

text = 'pantainya banyak sampah'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: pantainya banyak sampah | Label : Negative (95.880%)


In [32]:

text = 'dari parkiran ke pantainya jalan berbatu'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')



Text: dari parkiran ke pantainya jalan berbatu | Label : Negative (97.738%)
